In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
data_path = '/content/drive/My Drive/project/smart_manufacturing/csv_file'
train_log = pd.read_csv(os.path.join(data_path, 'train_log.csv'), low_memory = False)
# test_log = pd.read_csv(os.path.join(data_path, 'test_log.csv'), low_memory = False)

In [4]:
train_log_features = train_log.drop(['DATE', 'SUBSTRATE_ID', 'EVAPORATION_HALT'], axis = 1)
# test_log_features = test_log.drop(['DATE', 'SUBSTRATE_ID', 'EVAPORATION_HALT'], axis = 1)

In [5]:
train_log_features.head()

,P_ID,TIME,NUMBER_OF_PROCESSING,DEGREE_OF_VACUUM_IG,DEGREE_OF_VACUUM_PG,MASK_ID,SHUTTER,SRC3_SHUTTER_STATUS,PLC_THICKNESS_SP,PLC_THICKNESS_PV,PLC_SRC3_SHUTTER_OPEN_THICKNESS_SP,PLC_SRC3_SHUTTER_CLOSE_THICKNESS_SP,RAW_RATE_1,AVERAGE_RATE_1,SOURCE_POWER_OUTPUT_1,THICKNESS_1,STATE_1,STATE_TIME_1,ACCUMULATED_TIME_1,CRYSTAL_LIFE_1,CRYSTAL_REMAINING_1,CRYSTAL_POSITION_1,SENSOR_STATUS_1,FUNDAMENTAL_FREQUENCY_1,FUNDAMENTAL_ACTIVITY_1,DATA_LOG_1,RATE_DEVIATION_POWER_TREND_1,XXXXSIT_AVERAGE_RATE_1,STATE_BEFORE_STOP_1,CAUSE_OF_STOP_1,CHANNEL_ERRORS_WARNINGS_STATUS_1,CHANNEL_STATUS_1,XXXX_RATE_1,XXXX_POWER_1,XXXX_CRUCIBLE_1,XXXX_XTAL_1,XXXX_XTAL_USE_TIME_1,PREHEAT_RATE_1,PREHEAT_POWER_1,PREHEAT_CRUCIBLE_1,...,CRYSTAL_LIFE_3,CRYSTAL_REMAINING_3,CRYSTAL_POSITION_3,SENSOR_STATUS_3,FUNDAMENTAL_FREQUENCY_3,FUNDAMENTAL_ACTIVITY_3,DATA_LOG_3,RATE_DEVIATION_POWER_TREND_3,XXXXSIT_AVERAGE_RATE_3,STATE_BEFORE_STOP_3,CAUSE_OF_STOP_3,CHANNEL_ERRORS_WARNINGS_STATUS_3,CHANNEL_STATUS_3,XXXX_RATE_3,XXXX_POWER_3,XXXX_CRUCIBLE_3,XXXX_XTAL_3,XXXX_XTAL_USE_TIME_3,PREHEAT_RATE_3,PREHEAT_POWER_3,PREHEAT_CRUCIBLE_3,PREHEAT_XTAL_3,PREHEAT_XTAL_USE_TIME_3,CRUCIBLE31TEMP,CRUCIBLE32TEMP,CRUCIBLE33TEMP,CRUCIBLE34TEMP,CRUCIBLE35TEMP,CRUCIBLE36TEMP,CRUCIBLE37TEMP,INIT_PREHEAT_TIME_3,PREHEAT_START_TIME_3,CELL_CHANGE_TIME_3,CELL_CHANGE_CTRL_DLY_TIME_3,XXXX_USE_TIME_3,XXXX_COUNT_3,PLC_THICKNESS_3,MASK_CHANGE_COUNT_SP,MASK_CHANGE_COUNT_PV,champer
0,924,21:25:22,7,0.000064,0.016,00022SMH0023,CLOSE,CLOSE,250.0,0.0,0.0,0.0,1.0,1.0,0.00,0.0,6,1601.0,43594.0,1.0,1.0,1,0,5.965481,644.0,0,0,0.0,0,0,0,0,2.560,21.0,1,1,1435,0.0,0.0,3,...,0.0,1,1,0,5.973150,639.0,0,0,0.0,0,0,0,0,0.48,33.0,1,1,3121,0.0,0.0,3,1,166,339.1,28.0,23.8,24.9,27.4,0.0,0.0,1059,0,333,0,6010,3,0.0,0,5,5
1,924,21:25:32,7,0.000053,0.016,00022SMH0023,CLOSE,CLOSE,250.0,0.0,0.0,0.0,0.5,1.0,0.06,0.0,6,1611.0,43604.0,1.0,1.0,1,0,5.965473,644.0,0,0,0.0,0,0,0,0,2.560,20.9,1,1,1435,0.0,0.0,3,...,0.0,1,1,0,5.973148,638.0,0,0,0.0,0,0,0,0,0.48,32.7,1,1,3121,0.0,0.0,3,1,166,339.1,28.1,23.8,24.9,27.4,0.0,0.0,1059,0,333,0,6010,3,0.0,0,5,5
2,924,21:25:42,7,0.000048,0.016,00022SMH0023,CLOSE,CLOSE,250.0,0.0,0.0,0.0,0.5,1.0,0.03,0.0,6,1621.0,43614.0,1.0,1.0,1,0,5.965467,644.0,0,0,0.0,0,0,0,0,2.560,20.2,1,1,1435,0.0,0.0,3,...,0.0,1,1,0,5.973145,638.0,0,0,0.0,0,0,0,0,0.48,33.0,1,1,3121,0.0,0.0,3,1,166,339.1,28.0,23.8,24.9,27.4,0.0,0.0,1059,0,333,0,6010,3,0.0,0,5,5
3,924,21:25:52,7,0.000046,0.016,00022SMH0023,CLOSE,CLOSE,250.0,0.0,0.0,0.0,1.0,1.0,0.00,0.0,6,1631.0,43624.0,1.0,1.0,1,0,5.965460,644.0,0,0,0.0,0,0,0,0,2.624,19.4,1,1,1435,0.0,0.0,3,...,0.0,1,1,0,5.973143,639.0,0,0,0.0,0,0,0,0,0.48,33.0,1,1,3121,0.0,0.0,3,1,166,339.1,28.1,23.9,24.9,27.4,0.0,0.0,1059,0,332,0,6010,3,0.0,0,5,5
4,924,21:26:02,7,0.000044,0.016,00022SMH0023,CLOSE,CLOSE,250.0,0.0,0.0,0.0,1.0,1.0,0.00,0.0,6,1641.0,43634.0,1.0,1.0,1,0,5.965452,644.0,0,0,0.0,0,0,0,0,2.688,19.0,1,1,1435,0.0,0.0,3,...,0.0,1,1,0,5.973140,639.0,0,0,0.0,0,0,0,0,0.48,33.0,1,1,3121,0.0,0.0,3,1,166,339.1,28.1,23.9,24.9,27.4,0.0,0.0,1059,0,332,0,6010,3,0.0,0,5,5


In [6]:
def encoder(data):
    for col in data.columns:
        unique = pd.unique(data[col])
        if len(unique) < 10:
            fe = data.groupby(col).size()/len(data)
#             data.loc[:, col + '_encode'] = data[col].map(fe)
            data.loc[:, col] = data[col].map(fe)
#             data.drop([col], axis = 1, inplace = True)
        else:
            pass
    return np.array(data)

In [7]:
uniques = pd.unique(train_log_features['P_ID'])
time_length = []
for unique in uniques:
    target_pid = train_log_features.loc[train_log_features['P_ID'] == unique]
    time_length.append(len(target_pid))
print(np.min(time_length))
print(np.max(time_length))
print(np.std(time_length))
print(np.mean(time_length))

125
3492
202.76452840257554
292.2487381385019


In [8]:
def reshape_log(data):
    pids = pd.unique(data['P_ID'])
    log_features = np.zeros((1,292,146))
    for pid in pids:
        target_pid = data.loc[data['P_ID'] == pid]
        sort = target_pid.sort_values(by = ['TIME'])
        sort.drop(['TIME'], axis = 1, inplace = True)
        features = encoder(sort)
        if len(features) < 292:
          zeros_pad = np.zeros((292 - len(features), 146))
          features = np.vstack((features, zeros_pad))
          features = np.reshape(features, (1,292,146))
        else:
          features = features[:292]
          features = np.reshape(features, (1,292,146))
        log_features = np.concatenate((log_features, features))
    return log_features[1:]

In [9]:
log_features = reshape_log(train_log_features)

In [11]:
np.save('/content/drive/My Drive/project/smart_manufacturing/npy_file/train_log_array', log_features)